In [ ]:
import os
import sys
path_1 = os.path.dirname(os.getcwd())
path_2 = path_1 + '/' + 'CommonCodes/'
sys.path.insert(0, path_2)

import warnings
import operator
import numpy as np
import import_ipynb
import pandas as pd
from sklearn.svm import SVC
from collections import Counter
import matplotlib.pyplot as plt
import HelperFunctions as helper
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV


warnings.filterwarnings("ignore")

In [ ]:
_USE_FIRST_N_GENES_CONF = 40
_USE_FIRST_N_GENES_ALL = 100 

with open('./IBD_features.txt', 'r') as f:
    content = f.read()
    selected_genes = content.split(',')

selected_genes = selected_genes[:100000]

counts = Counter(selected_genes)
counts = dict(sorted(counts.items(), key=operator.itemgetter(1), reverse=True))
counts = list(counts.keys())[:_USE_FIRST_N_GENES_ALL]
selected_features = counts
selected_features.append('label')

train_datasets_names = ['GSE16879', 'GSE22619', 'GSE59071', 'GSE102133', 'GSE179285']
test_datasets_names = ['GSE9452', 'GSE36807' ,'GSE37283', 'GSE4183','GSE48958','GSE92415']

_test_datasets = []
for d in test_datasets_names:
    _test_datasets.append(pd.read_pickle(path_1+'/Datasets/'+d))

_train_datasets = []
for d in train_datasets_names:
    _train_datasets.append(pd.read_pickle(path_1+'/Datasets/'+d))

for i in range(len(_test_datasets)):
    _test_datasets[i] = _test_datasets[i].groupby(_test_datasets[i].columns, axis=1).agg(np.mean)[selected_features]
    _test_datasets[i].columns = np.arange(len(_test_datasets[i].columns))

for i in range(len(_train_datasets)):
    _train_datasets[i] = _train_datasets[i].groupby(_train_datasets[i].columns, axis=1).agg(np.mean)[selected_features]
    _train_datasets[i].columns = np.arange(len(_train_datasets[i].columns))

df_train = pd.concat(_train_datasets)

df_train = helper.oversample(df_train).sample(frac=1).reset_index(drop=True)

x_train, y_train = helper.datasetXY_S(df_train)

for i in range(len(_test_datasets)):
    x, y = helper.datasetXY_S(_test_datasets[i])
    x = pd.DataFrame(x.apply(helper.minmax_scaler, axis=1).values.tolist(), columns=x.columns)
    scaled_df = helper.datasetXY_M(x, y)
    _test_datasets[i] = scaled_df.copy()

x_train = pd.DataFrame(x_train.apply(helper.minmax_scaler, axis=1).values.tolist(), columns=x_train.columns)

In [ ]:
f1_scores = []
accuracies = []
confusions = []
recall_scores = []
specificity_scores = []

n_genes = [1,10,20,30,40,50,60,70,80,90,100]

for n in n_genes:
    
    x_train_f = x_train.iloc[:,:n]  

    rf_model = RandomForestClassifier(n_estimators=300)
    lr_model = LogisticRegression()
    svm_model = CalibratedClassifierCV(SVC(kernel='poly'), cv=5)

    rf_model.fit(x_train_f, y_train)
    lr_model.fit(x_train_f, y_train)
    svm_model.fit(x_train_f, y_train)


    n_genes_accs = []
    n_genes_f1 = []
    n_genes_recall = []
    n_genes_specificity = []
    
    for curr_test_set_index in range(len(_test_datasets)):

        x_test, y_test = helper.datasetXY_S(_test_datasets[curr_test_set_index])
        x_test_f = x_test.iloc[:,:n]
        
        rf_pred_proba = rf_model.predict_proba(x_test_f)
        lr_pred_proba = lr_model.predict_proba(x_test_f)
        svm_pred_proba = svm_model.predict_proba(x_test_f)

        ensemble_pred_probs = np.mean([rf_pred_proba, lr_pred_proba, svm_pred_proba], axis=0)

        ensemble_preds = helper.probToDummy(ensemble_pred_probs)[:,1]
        ensemble_preds = [int(x) for x in ensemble_preds]

        f1 = f1_score(y_test, ensemble_preds)
        acc = accuracy_score(y_test, ensemble_preds)

        n_genes_f1.append(f1)
        n_genes_accs.append(acc)                    

        if n == _USE_FIRST_N_GENES_CONF and y_test.value_counts().shape[0] == 2 :
            precision, recall, average_precision = helper.precision_recall(y_test, ensemble_pred_probs)
            classes = ['Control', 'Case']

            zom = [0,1,'micro']
            for z in zom:
                recall[z] = np.insert(recall[z], 0, 1)
            
            for z in zom:
                precision[z] = np.insert(precision[z], 0, 0)

            fig, ax = plt.subplots(figsize=(4.5, 4.5))
            
            ax.set_ylabel('Precision')
            ax.set_xlabel('Recall')
            ax.set_title(test_datasets_names[curr_test_set_index])

            ax.plot(recall["micro"], precision["micro"], label='Micro-average (area = {0:0.2f})'''.format(average_precision["micro"]))

            for i in range(2):
                ax.plot(recall[i], precision[i], label=f'{classes[i]} (area = {average_precision[i]:.2f})')

            ax = helper.init_precision_recall_plots(fig, ax, name = test_datasets_names[curr_test_set_index], save = True)

        cm = confusion_matrix(y_test, ensemble_preds)
        if cm.shape != (2, 2):
            case_or_control_num = cm[0,0]
            cm = [[0,0],[0,0]]
            if y_test[1] == 1 and ensemble_preds[1] == 1:
                cm = [[0,0],[0,case_or_control_num]]
            else:
                cm = [[case_or_control_num,0],[0,0]]
        
        cm = np.array(cm)

        TN = cm[0,0]
        FP = cm[0,1]
        FN = cm[1,0]
        TP = cm[1,1]

        if TP+FN!=0 and TN+FP==0:
            n_genes_recall.append(TP/(TP+FN))     

        if TN+FP!=0 and TP+FN==0:
            n_genes_specificity.append(TN/(TN+FP))
        
        if n == _USE_FIRST_N_GENES_CONF:
            confusions.append(cm.copy())
            
    if n_genes_recall:
        recall_scores.append(n_genes_recall.copy())
    
    if n_genes_specificity:
        specificity_scores.append(n_genes_specificity.copy())

    f1_scores.append(n_genes_f1.copy())
    accuracies.append(n_genes_accs.copy())

In [ ]:
recall_counter = 0
specificity_counter = 0

for i in range(len(test_datasets_names)):
    
    x = np.arange(len(n_genes))
    cm = confusions[i]
    fig, ax = plt.subplots(figsize=(4.5, 4.5))

    if cm[0,0] == 0 and cm[0,1] == 0:
        y = np.array(recall_scores)[:,recall_counter]
        recall_counter = recall_counter + 1
        ax.plot(x, y, label='Recall', marker='o', markerfacecolor='w', color='tab:purple')
        ax.set_ylabel('Recall')

    elif cm[1,0] == 0 and cm[1,1] == 0:
        y = np.array(specificity_scores)[:,specificity_counter]
        specificity_counter = specificity_counter + 1
        ax.plot(x, y, label='Specificity', marker='o', markerfacecolor='w', color='tab:green')
        ax.set_ylabel('Specificity') 

    elif cm[0,0]+cm[0,1] == cm[1,0]+cm[1,1]:
        y = np.array(accuracies)[:,i]
        ax.plot(x, y, label='Accuracy', marker='o', markerfacecolor='w', color='tab:blue')
        ax.set_ylabel('Accuracy') 

    else:
        y = np.array(accuracies)[:,i]
        ax.plot(x, y, label='Accuracy', marker='o', markerfacecolor='w', color='tab:blue')
        y = np.array(f1_scores)[:,i]
        ax.plot(x, y, label='F1-Score', marker='o', markerfacecolor='w', color='tab:orange')
        ax.set_ylabel('Score')

    ax.set_xlabel('# of Genes') 
    ax.set_title(test_datasets_names[i])
    ax.set_xticklabels([1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    ax.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    ax.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_xlim(-0.5, 10.5)
    ax.set_ylim(0.0, 1.05)
    ax.legend(loc='lower right', frameon=False)

    fig.savefig('Pictures/'+test_datasets_names[i]+'.png', format='png', dpi=500)
    helper.plot_confusion_matrix(_USE_FIRST_N_GENES_CONF, confusions[i], class_names= ['Control','Case'], name=test_datasets_names[i])

In [ ]:
controls_diagnosed_as_case = 0
total_controls = 0
cases_diagnosed_as_control = 0
total_cases = 0

for cm in confusions:
    controls_diagnosed_as_case = controls_diagnosed_as_case + cm[0,1]
    total_controls = total_controls + cm[0,0] + cm[0,1]
    cases_diagnosed_as_control = cases_diagnosed_as_control + cm[1,0]
    total_cases = total_cases + cm[1,0] + cm[1,1]


print(f'Total Cases: {total_cases}')
print(f'Cases misdiagnosed as controls: {cases_diagnosed_as_control}')
print(f'Total Controls: {total_controls}')
print(f'Controls misdiagnosed as controls: {controls_diagnosed_as_case}')